In [2]:
pip install jieba

  Using cached jieba-0.42.1-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [3]:
from __future__ import division
import sys
import argparse
import configparser
import json
import h5py as h5
import os
from os.path import isfile
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
from sklearn import linear_model
import pickle
import jieba
from tqdm import tqdm

from collections import Counter, defaultdict
from sklearn.utils import shuffle

In [4]:
config = configparser.ConfigParser()
my_config = '/home/jovyan/Project/Config/default.cfg'
with open('/home/jovyan/Project/Config/default.cfg', 'r') as f:
    config.read_file(f)

dsgv_home = config.get('DSGV-PATHS', 'dsgv_home')
preproc_path = dsgv_home + '/Preproc/PreprocOut/'
feats_path = dsgv_home + '/ExtractFeats/ExtractOut/'
wac_path = dsgv_home + '/WACs/'


# The first 3 features in the image feature Xs encode the corpus ID, image ID, region ID
ID_FEATS = 3

In [5]:
# Define classifier
classifier = linear_model.LogisticRegression
classf_params = {
    'penalty': 'l2',
    'warm_start': True,
    'solver': 'sag',
    'max_iter': 500
}

# Model description:
model = {
    'rcorp': 'flickr30k',        # ref corpus
    'cnn': 'rsn50-flatten_1',        # CNN used for vision feats
    'wrdl':  'min',            # wordlist: minimal n occurrences...
    'wprm':  40,               # ... 40 times
    'clsf':  'logreg-l2',      # logistic regression, l1 regularized
    'params': classf_params,
    'scaled': True,
    'nneg':  'balanced',       # maximum neg instances
    'nsrc':  'randmax',        # ... randomly selected
    'notes': ''
}

# Data

In [6]:
with open(preproc_path + 'flickr30k_split.json', 'r') as f:
    rc_splits = json.load(f)

In [7]:
# image features
with h5.File(feats_path + 'flickr_bbdf_rsn50-max_all.hdf5') as f:
    X = np.array(f["img_feats"])

In [8]:
def filter_X_by_filelist(X, filelist):
    tmp_df = pd.DataFrame(X)
    tmp_df = tmp_df[tmp_df.iloc[:,1].isin(filelist)]
    X_t = tmp_df.to_numpy()
    return X_t

def filter_refdf_by_filelist(refdf, filelist):
    return pd.merge(refdf, pd.DataFrame(filelist, columns=['image_id']))

In [9]:
X_t = filter_X_by_filelist(X, rc_splits['train'])

In [10]:
print(len(X_t))

258802


In [11]:
#referring expressions - CN
with open(preproc_path + 'flickr30k_objdf_cn.json.gz', 'r') as f:
    refdf_cn = json.load(f)
    refdf_cn = pd.DataFrame(refdf_cn)
refdf_train_cn = filter_refdf_by_filelist(refdf_cn, rc_splits['train'])

In [12]:
refdf_train_cn

,i_corpus,image_id,region_id,phrase,cat,phrase_cn
0,8,3187924573,99601,Two seagulls,animals,兩隻海鷗
1,8,3187924573,99604,water,scene,水
2,8,3187924573,99600,foreground bird,animals,前景鳥
3,8,3187924573,99603,object,other,目的
4,8,3187924573,99602,beak,bodyparts,喙
...,...,...,...,...,...,...
524860,8,408748500,154756,A child,people,一個孩子
524861,8,408748500,154757,a coat,clothing,一件外套
524862,8,408748500,154760,someone,people,某人
524863,8,408748500,154756,a boy,people,一個男孩


# Intermediate


In [13]:
def create_word2den_cn(refdf, refcol, regcol='region_id'):
    '''Given refdf, returns dict of occurences (id triples) of words from expressions.'''
    word2den = defaultdict(list)
    for _, row in refdf.iterrows():
        exprlist = jieba.cut(row[refcol])
        exprlist = '/'.join(exprlist)
        exprlist = exprlist.split('/')
        for word in exprlist:
            word_den_list = word2den[word].append((row['i_corpus'],
                                                   row['image_id'],
                                                   row[regcol]))
    return {k: list(set(v)) for k,v in word2den.items()}

In [14]:
%%time
word2den_cn = create_word2den_cn(refdf_train_cn, refcol='phrase_cn')

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.922 seconds.
Prefix dict has been built successfully.


CPU times: user 59.4 s, sys: 599 ms, total: 60 s
Wall time: 1min


In [15]:
with open(wac_path + 'ModelsOut/flickr_word2den_cn.pkl', 'wb') as f:
    pickle.dump(word2den_cn, f)
with open(wac_path + 'ModelsOut/flickr_word2den_cn.pkl', 'rb') as f:
    word2den_cn = pickle.load(f)

In [16]:
len(word2den_cn)

23363

Try a different Chinese text segmentation 

In [17]:
def create_word2den_cn2(refdf, refcol, regcol='region_id'):
    '''Given refdf, returns dict of occurences (id triples) of words from expressions.'''
    word2den = defaultdict(list)
    for _, row in refdf.iterrows():
        exprlist = jieba.cut(row[refcol],cut_all=True)
        exprlist = '/'.join(exprlist)
        exprlist = exprlist.split('/')

        for word in exprlist:
            word_den_list = word2den[word].append((row['i_corpus'],
                                                   row['image_id'],
                                                   row[regcol]))
    return {k: list(set(v)) for k,v in word2den.items()}

In [18]:
word2den_cn2 = create_word2den_cn2(refdf_train_cn, refcol='phrase_cn')

In [19]:
with open(wac_path + 'ModelsOut/flickr_word2den_cn2.pkl', 'wb') as f:
    pickle.dump(word2den_cn2, f)
with open(wac_path + 'ModelsOut/flickr_word2den_cn2.pkl', 'rb') as f:
    word2den_cn2 = pickle.load(f)

In [20]:
def make_X_id_index(X, id_feats=ID_FEATS):
    return dict(zip([tuple(e) for e in X[:,:id_feats].astype(int).tolist()], range(len(X))))

In [21]:
%%time
X_idx = make_X_id_index(X_t)

CPU times: user 738 ms, sys: 5.72 ms, total: 744 ms
Wall time: 748 ms


In [22]:
len(X_idx)

169943

length of X_idx : 169943

In [23]:
def make_mask_matrix(X, X_idx, word2den, wordlist):
    mask_matrix = []
    for this_word in wordlist:
        this_word_vec = np.zeros(len(X))
        if this_word in word2den:
          this_word_vec[[X_idx[i] for i in word2den[this_word] if i in X_idx]] = 1
        mask_matrix.append(this_word_vec)
    mask_matrix = np.array(mask_matrix, dtype=bool)
    return mask_matrix

In [24]:
wordlist_cn = word2den_cn.keys()

In [25]:
wordlist_cn2 = word2den_cn2.keys()

In [26]:
%%time
mask_matrix_cn = make_mask_matrix(X_t, X_idx, word2den_cn, word2den_cn.keys())

CPU times: user 12.8 s, sys: 11.3 s, total: 24.1 s
Wall time: 24.3 s


In [27]:
mask_matrix_cn.shape

(23363, 258802)

In [28]:
mask_matrix_cn2 = make_mask_matrix(X_t, X_idx, word2den_cn2, word2den_cn2.keys())

In [29]:
mask_matrix_cn2.shape

(10984, 258802)

In [30]:
np.savez(wac_path + 'flickr_mask_matrix_cn.npz', arr_0=mask_matrix_cn)
mask_matrix_cn = np.load(wac_path + 'flickr_mask_matrix_cn.npz')['arr_0']

In [31]:
np.savez(wac_path + 'flickr_mask_matrix_cn2.npz', arr_0=mask_matrix_cn2)
mask_matrix_cn2 = np.load(wac_path + 'flickr_mask_matrix_cn2.npz')['arr_0']

# Wordlist

In [32]:
min_freq = model['wprm']
counts_cn = mask_matrix_cn.sum(axis=1)
wordlist_cn = np.array(list(word2den_cn.keys()))[counts_cn > min_freq]

In [33]:
min_freq = model['wprm']
counts_cn2 = mask_matrix_cn2.sum(axis=1)
wordlist_cn2 = np.array(list(word2den_cn2.keys()))[counts_cn2 > min_freq]

In [34]:
def get_X_for_word(X, word2den, mask_matrix, word, neg_max):
    if word not in word2den:
        print ("Error!! No mask available for this word! (%s)" % (word))
        return None
    this_mask = mask_matrix[list(word2den.keys()).index(word)]
    X_pos = X[this_mask, ID_FEATS:]
    y_pos = np.ones(len(X_pos), dtype=int)

    if type(neg_max) is int or type(neg_max) is str:
        if neg_max == 0:
            return X_pos, y_pos
        if neg_max == 'balanced':
            neg_max = len(y_pos)
    
        neg_indx = np.arange(mask_matrix.shape[1])[~this_mask]
        np.random.shuffle(neg_indx)
        X_neg = X[neg_indx[:neg_max], ID_FEATS:]
    else:
        X_neg = neg_max
    y_neg = np.zeros(len(X_neg), dtype=int)

    X_out = np.concatenate([X_pos, X_neg], axis=0)
    y_out = np.concatenate([y_pos, y_neg])
    return shuffle(X_out, y_out)

In [35]:
def make_train_for_word(X, word2den, mask_matrix, word, neg_max=20000):
    if word not in word2den:
        print("Error!! No mask available for this word! (%s)" % (word))
        return None
    this_mask = mask_matrix[list(word2den.keys()).index(word)]
    X_pos = X[this_mask, ID_FEATS:]
    y_pos = np.ones(len(X_pos), dtype=int)
    
    neg_indx = np.arange(mask_matrix.shape[1])[~this_mask]
    np.random.shuffle(neg_indx)
    # X_neg = X[neg_indx[:neg_max], ID_FEATS:]
    X_neg = X[neg_indx[:len(y_pos)], ID_FEATS:]
    y_neg = np.zeros(len(X_neg), dtype=int)

    X_out = np.concatenate([X_pos, X_neg], axis=0)
    y_out = np.concatenate([y_pos, y_neg])
    return shuffle(X_out, y_out)

# Train

Train the baseline wac

For logistic regression, we use l2 penalty and 'sag' solver.
Use this as baseline model.

Segmentation - accurate mode

In [29]:
%%time
wacs_cn = []
for this_word in tqdm(wordlist_cn):
    X_this_w, y_this_w = get_X_for_word(X_t, word2den_cn, mask_matrix_cn, this_word, neg_max='balanced')
    print(this_word, X_this_w.shape[0])
    classifier = linear_model.LogisticRegression(penalty='l2', warm_start=True,solver='sag')
    this_wac = classifier.fit(X_this_w, y_this_w)
    wacs_cn.append((this_word, y_this_w.sum(), len(X_this_w),this_wac))

  0%|          | 0/1378 [00:00<?, ?it/s]

兩 1076


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
  0%|          | 1/1378 [00:02<56:26,  2.46s/it]

隻 6258


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
  0%|          | 2/1378 [00:15<3:21:49,  8.80s/it]

...

蝴蝶 82
棋盤 88


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
100%|█████████▉| 1376/1378 [33:59<00:00,  4.79it/s]/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
100%|█████████▉| 1377/1378 [33:59<00:00,  4.75it/s]

色連 92
抗議者 124


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
100%|██████████| 1378/1378 [34:00<00:00,  1.48s/it]

CPU times: user 33min 39s, sys: 9.38 s, total: 33min 49s
Wall time: 34min


In [36]:
with open(wac_path + 'ModelsOut/flickr_wacs_cn.pkl', 'wb') as f:
    pickle.dump(wacs_cn, f)
with open(wac_path + 'ModelsOut/flickr_wacs_cn.pkl', 'rb') as f:
    wacs_cn = pickle.load(f)

In [30]:
len(wacs_cn)

1378

Segmentation - full mode

In [ ]:
%%time
wacs_cn2 = []
for this_word in tqdm(wordlist_cn2):
    X_this_w, y_this_w = get_X_for_word(X_t, word2den_cn2, mask_matrix_cn2, this_word, neg_max='balanced')
    print(this_word, X_this_w.shape[0])
    classifier = linear_model.LogisticRegression(penalty='l2', warm_start=True,solver='sag')
    this_wac = classifier.fit(X_this_w, y_this_w)
    wacs_cn2.append((this_word, y_this_w.sum(), len(X_this_w),this_wac))

  0%|          | 0/1507 [00:00<?, ?it/s]

兩 14094


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
  0%|          | 1/1507 [00:24<10:03:18, 24.04s/it]

隻 6258


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
  0%|          | 2/1507 [00:33<6:30:12, 15.56s/it] 

...

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
 79%|███████▉  | 1196/1507 [41:12<01:50,  2.82it/s]

經 140


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
 79%|███████▉  | 1197/1507 [41:12<01:45,  2.93it/s]

一架 178


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
 79%|███████▉  | 1198/1507 [41:13<01:54,  2.70it/s]

In [37]:
with open(wac_path + 'ModelsOut/flickr_wacs_cn2.pkl', 'wb') as f:
    pickle.dump(wacs_cn2, f)
with open(wac_path + 'ModelsOut/flickr_wacs_cn2.pkl', 'rb') as f:
    wacs_cn2 = pickle.load(f)

In [38]:
len(wacs_cn2)

1507

# Evaluation

In [36]:
def get_X_for_word(X, word2den, mask_matrix, word, neg_max):
    if word not in word2den:
        print("Error!! No mask available for this word! (%s)" % (word))
        return None
    this_mask = mask_matrix[list(word2den.keys()).index(word)]
    X_pos = X[this_mask, ID_FEATS:]
    y_pos = np.ones(len(X_pos), dtype=int)
    
    neg_indx = np.arange(mask_matrix.shape[1])[~this_mask]
    np.random.shuffle(neg_indx)
    
    if neg_max == 'balanced':
      neg_max = len(y_pos)
    X_neg = X[neg_indx[:neg_max], ID_FEATS:]
    y_neg = np.zeros(len(X_neg), dtype=int)

    X_out = np.concatenate([X_pos, X_neg], axis=0)
    y_out = np.concatenate([y_pos, y_neg])
    return shuffle(X_out, y_out)

## Training accuracy

Segmentation - accurate mode

In [ ]:
acc_cn = []
for this_word, npos, _, this_clsf in tqdm(wacs_cn):
    X_tst, y_tst = get_X_for_word(X_t, word2den_cn, mask_matrix_cn, this_word, neg_max='balanced')
    score = this_clsf.score(X_tst, y_tst)
    acc_cn.append(score)

100%|██████████| 1378/1378 [00:47<00:00, 29.12it/s]


In [ ]:
#training accuracy (CN)
print('average:',sum(acc_cn)/len(acc_cn))

average: 0.9077675182751802


Segmentation - full mode

In [32]:
acc_cn2 = []
for this_word, npos, _, this_clsf in tqdm(wacs_cn2):
    X_tst, y_tst = get_X_for_word(X_t, word2den_cn2, mask_matrix_cn2, this_word, neg_max='balanced')
    score = this_clsf.score(X_tst, y_tst)
    acc_cn2.append(score)

100%|██████████| 1507/1507 [04:52<00:00,  5.16it/s]


In [33]:
print('average:',sum(acc_cn2)/len(acc_cn2))

average: 0.8918474868553767


Performance on training data (!) unsuprisingly pretty good...

## Test CN WACs on the test set (accurate mode)
Evaluation on segmentation 1

In [29]:
rc_all_test = rc_splits['val'] + rc_splits['test']
X_ts = filter_X_by_filelist(X, rc_all_test)
refdf_test = filter_refdf_by_filelist(refdf_cn, rc_all_test)

In [34]:
word2den_ts = create_word2den_cn(refdf_test, refcol='phrase_cn')
X_idx_ts = make_X_id_index(X_ts)
mask_matrix_ts = make_mask_matrix(X_ts, X_idx_ts, word2den_ts, word2den_ts.keys())

In [86]:
test_acc = []
test_words = []
for this_word, npos, _, this_clsf in tqdm(wacs_cn):
    try:
        X_tst, y_tst = get_X_for_word(X_ts, word2den_ts, mask_matrix_ts, this_word, neg_max='balanced')
        test_acc.append(this_clsf.score(X_tst, y_tst))
        test_words.append((this_word,this_clsf.score(X_tst, y_tst)))
    except:
        print('this word:',this_word)

  5%|▍         | 68/1378 [00:07<01:49, 11.95it/s]

Error!! No mask available for this word! (黑皮)
this word: 黑皮


 30%|███       | 416/1378 [00:23<00:20, 47.82it/s]

this word: 紅磚


 31%|███       | 423/1378 [00:23<00:21, 43.51it/s]

...

Error!! No mask available for this word! (斑點)
this word: 斑點
Error!! No mask available for this word! (保齡)
this word: 保齡
Error!! No mask available for this word! (莫霍克)
this word: 莫霍克
Error!! No mask available for this word! (保齡球)
this word: 保齡球


 90%|████████▉ | 1240/1378 [00:31<00:00, 264.74it/s]

Error!! No mask available for this word! (部落)
this word: 部落
this word: 起重
this word: 木筏
Error!! No mask available for this word! (原住民)
this word: 原住民


100%|██████████| 1378/1378 [00:31<00:00, 43.65it/s] 

Error!! No mask available for this word! (白馬)
this word: 白馬
Error!! No mask available for this word! (燃燒)
this word: 燃燒
Error!! No mask available for this word! (鴿子)
this word: 鴿子


In [91]:
for this_word, npos, _, this_clsf in wacs[:10]:
    X_tst, y_tst = get_X_for_word(X_ts, word2den_ts, mask_matrix_ts, this_word, neg_max='balanced')
    # print(this_word, npos, '\t',
    # X_tst, y_tst)
    print(this_word, npos)
    print(this_clsf.score(X_tst, y_tst))

兩 538
0.7916666666666666
隻 3129
0.8666666666666667
水 1718
0.8608247422680413
鳥 152
0.9
一 3698
0.7050209205020921
某物 602
0.6666666666666666
它 611
0.7272727272727273
的 44289
0.6439367914805909
大白 58
0.6666666666666666
一個 38411
0.7071260767423649


In [87]:
print('average:',sum(test_acc)/len(test_acc))

average: 0.8193471594459746


In [38]:
test_words_sorted = sorted(test_words, key=lambda x: x[1])
test_words_sorted

[('自', 0.0),
 ('針織', 0.0),
 ('站立', 0.16666666666666666),
 ('夫', 0.25),
 ('布', 0.25),
 ('彼此', 0.3333333333333333),
 ('白種', 0.375),
 ('鏡頭', 0.375),
 ('全部', 0.4),
 ('多', 0.4),
 ('相同', 0.4),
 ('這些', 0.4166666666666667),
 ('姿勢', 0.42857142857142855),
 ('中國', 0.4375),
 ('淺色', 0.4375),
 ('是', 0.4444444444444444),
 ('大白', 0.5),
 ('膚', 0.5),
 ('之一', 0.5),
 ('模型', 0.5),
 ('新', 0.5),
 ('風格', 0.5),
 ('我', 0.5),
 ('對方', 0.5),
 ('面', 0.5),
 ('一條紅', 0.5),
 ('表演者', 0.5),
 ('小牛', 0.5),
 ('像', 0.5),
 ('背景', 0.5),
 ('宗教', 0.5),
 ('上', 0.5),
 ('偽裝', 0.5),
 ('輕便', 0.5),
 ('隊員', 0.5),
 ('較', 0.5),
 ('軍', 0.5),
 ('只有', 0.5),
 ('各樣', 0.5),
 ('時髦', 0.5),
 ('沉重', 0.5),
 ('人物', 0.5),
 ('高', 0.5),
 ('彩繪', 0.5),
 ('主人', 0.5),
 ('透明', 0.5),
 ('5', 0.5),
 ('織物', 0.5),
 ('橡膠', 0.5),
 ('女嬰', 0.5),
 ('形象', 0.5),
 ('蓬', 0.5),
 ('表情', 0.5),
 ('多彩多姿', 0.5),
 ('雜誌', 0.5),
 ('前', 0.5),
 ('附近', 0.5),
 ('吊帶', 0.5),
 ('滑稽', 0.5),
 ('老夫妻', 0.5),
 ('中國人', 0.5),
 ('乾草', 0.5),
 ('旋轉', 0.5),
 ('所有', 0.5),
 ('罐頭', 0.5),
 ('多彩的', 0.5

In [39]:
this_wac = wacs_cn[0][3]

In [40]:
this_wac.intercept_

array([-8.85429484e-05])

In [41]:
np.stack([np.append(this_wac.coef_, this_wac.intercept_) \
          for this_wac in [w[3] for w in wacs_cn]]).shape

(1378, 2056)

In [42]:
sys.stdout.flush()

In [43]:
wl = [e[:-1] for e in wacs_cn]

In [61]:
weight_matrix_cn = np.stack([np.append(this_wac.coef_,
                                    this_wac.intercept_)
                              for this_wac in [w[3] for w in wacs_cn]])

In [63]:
weight_matrix_cn.shape

(1378, 2056)

In [64]:
np.savez_compressed(wac_path + 'ModelsOut/weight_flickr_cn.npz', weight_matrix_cn)

In [76]:
weightmatrix_ld = np.load(wac_path + 'ModelsOut/weight_flickr_cn.npz')

In [77]:
weightmatrix = weightmatrix_ld['arr_0']

In [78]:
weightmatrix.shape

(1207, 2056)

## Test CN WACs on the test set (full mode)
Evaluation on segmentation 2

In [52]:
rc_all_test = rc_splits['val'] + rc_splits['test']
X_ts = filter_X_by_filelist(X, rc_all_test)
refdf_test = filter_refdf_by_filelist(refdf_cn, rc_all_test)

In [53]:
word2den_ts = create_word2den_cn2(refdf_test, refcol='phrase_cn')
X_idx_ts = make_X_id_index(X_ts)
mask_matrix_ts = make_mask_matrix(X_ts, X_idx_ts, word2den_ts, word2den_ts.keys())

In [46]:
test_acc2 = []
test_words2 = []
for this_word, npos, _, this_clsf in tqdm(wacs_cn2):
    try:
        X_tst, y_tst = get_X_for_word(X_ts, word2den_ts, mask_matrix_ts, this_word, neg_max='balanced')
        test_acc.append(this_clsf.score(X_tst, y_tst))
        test_words.append((this_word,this_clsf.score(X_tst, y_tst)))
    except:
        print('this word:',this_word)

 17%|█▋        | 254/1507 [00:23<01:14, 16.84it/s]

this word: 揮


 35%|███▌      | 532/1507 [00:35<00:15, 61.07it/s]

this word: 興


 38%|███▊      | 568/1507 [00:35<00:18, 51.97it/s]

...

this word: 民
Error!! No mask available for this word! (莫霍克)
this word: 莫霍克
Error!! No mask available for this word! (霍克)
this word: 霍克
this word: 譜


 93%|█████████▎| 1396/1507 [00:43<00:00, 182.67it/s]

Error!! No mask available for this word! (部落)
this word: 部落
this word: 起重
this word: 木筏
Error!! No mask available for this word! (原住民)
this word: 原住民
Error!! No mask available for this word! (住民)
this word: 住民


100%|██████████| 1507/1507 [00:44<00:00, 34.13it/s] 

Error!! No mask available for this word! (鴿)
this word: 鴿
Error!! No mask available for this word! (瘦)
this word: 瘦


In [48]:
print('average:',sum(test_acc2)/len(test_acc2))

average: 0.784375453681655


In [49]:
this_wac = wacs_cn2[0][3]

In [50]:
this_wac.intercept_

array([-0.00746788])

In [51]:
np.stack([np.append(this_wac.coef_, this_wac.intercept_) \
          for this_wac in [w[3] for w in wacs_cn2]]).shape

(1507, 2056)

In [52]:
sys.stdout.flush()

In [54]:
wl = [e[:-1] for e in wacs_cn2]

In [55]:
wl

[('兩', 7047, 14094),
 ('隻', 3129, 6258),
 ('海', 660, 1320),
 ('水', 2249, 4498),
 ('鳥', 205, 410),
 ('一', 55634, 111268),
 ('白', 2179, 4358),
 ('某物', 602, 1204),
 ('它', 651, 1302),
 ('的', 46843, 93686),
 ('大白', 67, 134),
 ('條', 6681, 13362),
 ('魚', 314, 628),
 ('個', 48055, 96110),
 ('蹣', 423, 846),
 ('跚', 423, 846),
 ('學', 1034, 2068),
 ('步', 696, 1392),
 ('男孩', 4096, 8192),
 ('頂', 2678, 5356),
 ('藍', 6983, 13966),
 ('色', 15283, 30566),
 ('棒球', 695, 1390),
 ('棒球帽', 272, 544),
 ('單', 594, 1188),
 ('年幼', 832, 1664),
 ('孩子', 4861, 9722),
 ('手', 3875, 7750),
 ('成年', 521, 1042),
 ('成年人', 397, 794),
 ('某人', 588, 1176),
 ('梯子', 124, 248),
 ('小男孩', 2343, 4686),
 ('帽子', 2074, 4148),
 ('兒', 990, 1980),
 ('童', 293, 586),
 ('公', 592, 1184),
 ('園', 658, 1316),
 ('三', 2270, 4540),
 ('朋友', 457, 914),
 ('船', 584, 1168),
 ('明亮', 194, 388),
 ('岩石', 536, 1072),
 ('海岸', 91, 182),
 ('赤膊', 89, 178),
 ('男子', 2644, 5288),
 ('槳', 105, 210),
 ('支', 107, 214),
 ('撐', 51, 102),
 ('男人', 13220, 26440),
 ('另外', 243, 

In [56]:
weight_matrix = np.stack([np.append(this_wac.coef_,
                                    this_wac.intercept_)
                              for this_wac in [w[3] for w in wacs_cn2]])

In [57]:
weight_matrix.shape

(1507, 2056)

In [58]:
np.savez_compressed(wac_path + 'ModelsOut/weight_flickr_cn2.npz', weight_matrix)

In [61]:
weightmatrix_ld = np.load(wac_path + 'ModelsOut/weight_flickr_cn2.npz')

In [62]:
weightmatrix = weightmatrix_ld['arr_0']

In [63]:
weightmatrix.shape

(1507, 2056)

#  Error  analysis
Take segmentation 1 for example

In [60]:
test_words_sorted = sorted(test_words, key=lambda x: x[1])
word_count = np.array((counts_cn,np.array(list(word2den_cn.keys()))))
sort_arr = sorted(word_count.transpose(), key=lambda x: int(x[0]))
words = [a for a in sort_arr if int(a[0])>40]
word_prob = pd.DataFrame(test_words_sorted,columns = ['word','prob'])
count_word = pd.DataFrame(words, columns = ['count','word'])
word_prob_count = word_prob.merge(count_word, how='inner', on='word')

In [62]:
word_prob_count[:30]

,word,prob,count
0,自,0.000000,52
1,針織,0.000000,81
2,站立,0.166667,43
3,夫,0.250000,49
4,布,0.250000,57
5,彼此,0.333333,45
6,白種,0.375000,71
7,鏡頭,0.375000,57
8,全部,0.400000,205
9,多,0.400000,171


# Reference

User defined functions:

https://github.com/clp-research/clp-vision

Chinese segmentation:

https://pypi.org/project/jieba/

Logistic Regression model:

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html